# Circuit simulation examples

In [1]:
import obi_one as obi

In [2]:
circuit_path_prefix = "/Users/james/Documents/obi/additional_data/O1_data/O1_data/"
# circuit_path_prefix = "/Users/pokorny/Data/Circuits/"

### Loading two circuits

In [3]:
# circuit = obi.Circuit(name="ToyCircuit-S1-6k", path=circuit_path_prefix + "ToyCircuit-S1-6k/circuit_config.json")
circuit = obi.Circuit(name="O1", path=circuit_path_prefix + "circuit_config.json")
print(f"Circuit '{circuit}' with {circuit.sonata_circuit.nodes.size} neurons and {circuit.sonata_circuit.edges.size} synapses")

Circuit 'O1' with 785632 neurons and 560631825 synapses


In [4]:
# circuit2 = obi.Circuit(name="ToyCircuit-S1-6k-duplicate", path="/Users/pokorny/Data/Circuits/ToyCircuit-S1-6k/circuit_config.json")
# circuit2 = obi.Circuit(name="nbS1-HEX0-beta", path=circuit_path_prefix + "ToyCircuit-S1-6k/circuit_config.json")
circuit2 = obi.Circuit(name="O1_2", path=circuit_path_prefix + "circuit_config.json")
print(f"Circuit '{circuit2}' with {circuit2.sonata_circuit.nodes.size} neurons and {circuit2.sonata_circuit.edges.size} synapses")

Circuit 'O1_2' with 785632 neurons and 560631825 synapses


### Defining the circuit simulation

In [5]:
# Simulation init
sim_neuron_set = obi.PredefinedNeuronSet(node_set="Layer1")
sim_duration = 3000 # ms
poisson_stim_duration = 800 # ms
poisson_stim_frequency = 20 # Hz
simulations_initialize = obi.SimulationsForm.Initialize(
    circuit=[circuit, circuit2],
    node_set=obi.NeuronSetReference(block_name="L1All", block_dict_name="neuron_sets"),
    simulation_length=sim_duration,
)



# Stimuli
property_neuron_set = obi.PropertyNeuronSet(
    property_filter=[obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": [2, 3], "synapse_class": ["INH"]}),
                     obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": [1, 2, 3], "synapse_class": ["EXC"]})],
)
stim_neuron_set = obi.PredefinedNeuronSet(node_set="Layer1", random_sample=[10, 20])
stim_times = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=1000.0)  # in ms!!
current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=obi.TimestampsReference(block_name="StimTimes", block_dict_name="timestamps"),
                                                           duration=5.0, 
                                                           neuron_set=obi.NeuronSetReference(block_name="L1Stim", block_dict_name="neuron_sets"), 
                                                           amplitude=[0.2, 0.5])
sync_current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=obi.TimestampsReference(block_name="StimTimes", block_dict_name="timestamps"), 
                                                                duration=100.0, 
                                                                neuron_set=obi.NeuronSetReference(block_name="L1Stim", block_dict_name="neuron_sets"), 
                                                                amplitude=0.1)
poisson_input = obi.PoissonSpikeStimulus(timestamps=obi.TimestampsReference(block_name="StimTimes", block_dict_name="timestamps"),
                                                        stim_duration=poisson_stim_duration,
                                                        frequency=poisson_stim_frequency,
                                                        neuron_set=obi.NeuronSetReference(block_name="L1Stim", block_dict_name="neuron_sets"), 
                                                        )

# Recordings
rec_neuron_set = obi.PredefinedNeuronSet(node_set="Layer1", random_sample=100)
v_recording = obi.SomaVoltageRecording(start_time=0.0, 
                                       end_time=sim_duration, 
                                       neuron_set=obi.NeuronSetReference(block_name="L1Rec", block_dict_name="neuron_sets")
                                        )

"""
Fill form with Blocks
"""
simulations_form = obi.SimulationsForm(
                                    initialize=simulations_initialize,
                                    info=obi.Info(campaign_name="Example Simulation", campaign_description="This is an example simulation."),
                                    neuron_sets={"L1All": sim_neuron_set, "L1Stim": stim_neuron_set, "L1Rec": rec_neuron_set, "PropertyNeuronSet": property_neuron_set},
                                    timestamps={"StimTimes": stim_times},
                                    stimuli={"CurrentStimulus": current_stimulus, "SyncCurrentStimulus": sync_current_stimulus, "PoissonInputStimulus": poisson_input},
                                    recordings={"SomaVoltRec": v_recording},
                                    )

In [6]:
simulations_form.model_dump(mode="json")

{'type': 'SimulationsForm',
 'timestamps': {'StimTimes': {'type': 'RegularTimestamps',
   'start_time': 0.0,
   'number_of_repetitions': 3,
   'interval': 1000.0}},
 'stimuli': {'CurrentStimulus': {'type': 'ConstantCurrentClampSomaticStimulus',
   'timestamps': {'type': 'TimestampsReference',
    'block_dict_name': 'timestamps',
    'block_name': 'StimTimes'},
   'delay': 0.0,
   'duration': 5.0,
   'neuron_set': {'type': 'NeuronSetReference',
    'block_dict_name': 'neuron_sets',
    'block_name': 'L1Stim'},
   'represents_physical_electrode': False,
   'amplitude': [0.2, 0.5]},
  'SyncCurrentStimulus': {'type': 'ConstantCurrentClampSomaticStimulus',
   'timestamps': {'type': 'TimestampsReference',
    'block_dict_name': 'timestamps',
    'block_name': 'StimTimes'},
   'delay': 0.0,
   'duration': 100.0,
   'neuron_set': {'type': 'NeuronSetReference',
    'block_dict_name': 'neuron_sets',
    'block_name': 'L1Stim'},
   'represents_physical_electrode': False,
   'amplitude': 0.1},
  '

In [7]:
grid_scan = obi.GridScan(form=simulations_form, output_root='../../obi-output/circuit_simulations/grid_scan')
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
# grid_scan.coordinate_instances(display=True)
grid_scan.execute(processing_method='generate')

[2025-06-11 20:49:14,250] INFO: 
MULTIPLE VALUE PARAMETERS
[2025-06-11 20:49:14,250] INFO: stimuli.CurrentStimulus.amplitude: [0.2, 0.5]
[2025-06-11 20:49:14,251] INFO: neuron_sets.L1Stim.random_sample: [10, 20]
[2025-06-11 20:49:14,251] INFO: neuron_sets.PropertyNeuronSet.property_filter: [layer=2,3,synapse_class=INH, layer=1,2,3,synapse_class=EXC]
[2025-06-11 20:49:14,251] INFO: initialize.circuit: [Circuit(type='Circuit', name='O1', path='/Users/james/Documents/obi/additional_data/O1_data/O1_data/circuit_config.json', matrix_path=None), Circuit(type='Circuit', name='O1_2', path='/Users/james/Documents/obi/additional_data/O1_data/O1_data/circuit_config.json', matrix_path=None)]
[2025-06-11 20:49:14,251] INFO: 
COORDINATE PARAMETERS
[2025-06-11 20:49:14,252] INFO: stimuli.CurrentStimulus.amplitude: 0.2, neuron_sets.L1Stim.random_sample: 10, neuron_sets.PropertyNeuronSet.property_filter: layer=2,3,synapse_class=INH, initialize.circuit: O1
[2025-06-11 20:49:14,252] INFO: stimuli.Current

{0: None,
 1: None,
 2: None,
 3: None,
 4: None,
 5: None,
 6: None,
 7: None,
 8: None,
 9: None,
 10: None,
 11: None,
 12: None,
 13: None,
 14: None,
 15: None}

In [8]:
# Deserialization
grid_scan_ds = obi.deserialize_obi_object_from_json_file("../../obi-output/circuit_simulations/grid_scan/run_scan_config.json")